In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('crime_data.csv')
df.head()

,year,timestop,recstat,inout,trhsloc,crimsusp,typeofid,explnstp,othpers,sumissue,...,sex,race,age,weight,build,city,weapons,physical_force,height,arstmade
0,2015,315,1,O,P,other,V,Y,N,N,...,M,W,33,190,M,BROOKLYN,N,N,71,N
1,2015,1747,A,O,P,other,V,Y,N,N,...,M,B,14,140,T,MANHATTAN,N,Y,68,N
2,2015,1745,1,O,P,MISD,V,Y,N,N,...,M,B,14,140,T,MANHATTAN,N,Y,63,N
3,2015,1745,1,O,P,other,V,Y,Y,N,...,M,B,14,180,M,MANHATTAN,N,N,69,N
4,2015,1745,1,O,P,other,V,Y,Y,N,...,M,B,13,160,M,MANHATTAN,N,Y,70,N


In [3]:
df.shape

(30895, 49)

In [4]:
target = ['arstmade']

In [5]:
pd.crosstab(df.race, df.arstmade)

arstmade,N,Y
race,,
B,13377,3050
Q,5291,1607
W,2792,529
Z,3418,831


In [6]:
null_val_count = []
for col in df.columns:
    null_val_count.append(df[col].isna().any())
res = True in (x==True for x in null_val_count)
res

False

In [7]:
def get_num_cols(data):
    cols = data.columns
    num_cols = data._get_numeric_data().columns
    return num_cols

In [8]:
def get_categorical_cols(data):
    cols = data.columns
    num_cols = data._get_numeric_data().columns
    cat_cols = list(set(cols) - set(num_cols))
    return cat_cols

In [9]:
def perform_T_test(data):
    num_cols = get_num_cols(data)
    
    arrY = data[data['arstmade']=='Y']
    arrN = data[data['arstmade']=='N']
    
    from scipy.stats import ttest_ind
    t_scores = {}
    for col in num_cols:
        if col not in t_scores:
            t_scores[col] = []
            t_scores[col].append(ttest_ind(arrY[col], arrN[col]).pvalue)
    #print(t_scores)
    return t_scores

In [10]:
from sklearn.preprocessing import LabelEncoder
def perform_chi_sq_test(data):
    cat_cols = get_categorical_cols(data)
    #return cat_cols
    label_encoder = LabelEncoder()
    for col in cat_cols:
        data[col] = label_encoder.fit_transform(data[col])
        
    X = data.drop('arstmade', axis=1)
    y = data['arstmade']
    
    from sklearn.feature_selection import chi2
    chi_score = chi2(X,y)
    p_vals = pd.Series(chi_score[1], index = X.columns)
    p_vals.sort_values(ascending=False, inplace=True)
    return p_vals

In [11]:
data = df

In [12]:
t_scores = perform_T_test(data)

In [13]:
t_scores

{'year': [6.139046286422341e-13],
 'timestop': [0.002765283856341679],
 'age': [0.11365829441118332],
 'weight': [6.742319190393864e-05],
 'height': [0.0001919002963451546]}

In [14]:
chi_scores = perform_chi_sq_test(data)

In [15]:
chi_scores.round(5).tail(10)

inout             0.0
city              0.0
rf_othsw          0.0
recstat           0.0
physical_force    0.0
contrabn          0.0
searched          0.0
sb_other          0.0
weapons           0.0
timestop          0.0
dtype: float64

In [16]:
discard_features = ['year','explnstp','crimsusp','ac_other','cs_bulge','ac_rept','height','weight','timestop','build','sex']

In [17]:
for val in discard_features:
    data = data.drop([val], axis=1)
data.shape

(30895, 38)

In [ ]:
data.to_csv('crime_data_updated_features.csv')

In [ ]:
df = pd.read_csv('crime_data_final.csv')
df.head()